In [1]:
import os
import pandas as pd
import platform
import selenium
import pickle
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
operating_system = platform.system()

if operating_system == 'Linux':
    webdriver_path = os.path.join('chromedriver_linux64', 'chromedriver')
elif operating_system == 'Windows':
    webdriver_path = os.path.join('chromedriver_win32', 'chromedriver.exe')

service = Service(executable_path = webdriver_path)

In [20]:
driver = webdriver.Chrome(service = service)
web_driver_wait = WebDriverWait(driver, 20)

for i in range (1, 50):
    link = f'https://genius.com/tags/mpb/all?page={i}'
    driver.get(link)
    
    '''while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        if not (len(driver.find_elements(By.XPATH, "//a[contains(text(), 'more')]"))) > 0:
            break'''
    
    song_elements_id = '//span[@class="title_with_artists"]'
    song_elements = web_driver_wait.until(EC.presence_of_all_elements_located((By.XPATH, song_elements_id)))
    
    for j in range(len(song_elements)):
        print(f'Página {i}, música número {j + 1}.')
        song = song_elements[j]
        parent = song.find_element(By.XPATH, '..')
        song_title = song.find_element(By.CLASS_NAME, 'song_title').text
        
        artists_element = song.find_element(By.CLASS_NAME, 'artist_name')
        artist_name = artists_element.find_element(By.CLASS_NAME, 'primary_artist_name').text
        
        featured_artists_list = []
        try:
            featured_artists_names = artists_element.find_elements(By.CLASS_NAME, 'featured_artist_name')
            for featured_artist_name in featured_artists_names:
                featured_artists_list.append(featured_artist_name.text)
        except:
            pass

        lyrics_link = parent.get_attribute('href')
        driver.execute_script(f'window.open("{lyrics_link}","_blank");')
        driver.switch_to.window(driver.window_handles[1])
        
        lyrics = ''
        try:
            containers_with_lyrics_id = '//div[@data-lyrics-container="true"]'
            containers_with_lyrics = web_driver_wait.until(EC.presence_of_all_elements_located((By.XPATH, containers_with_lyrics_id)))
            for container in containers_with_lyrics:
                current_lyrics = container.text
                lyrics += current_lyrics
        except:
            pass
        
        try:
            expand_buttons = driver.find_elements(By.XPATH, "//button[contains(text(), 'Expand')]")
            for expand_button in expand_buttons:
                expand_button.click()
        except:
            pass
        
        
        release_date = ''
        try:
            release_div = driver.find_element(By.XPATH, "//div[contains(text(), 'Release Date')]")
            release_div_parent = release_div.find_element(By.XPATH, '..')
            release_date = release_div_parent.text
        except:
            pass
        
        writers = ''
        writers_div = driver.find_element(By.XPATH, "//div[contains(text(), 'Written By')]")
        writers_div_parent = writers_div.find_element(By.XPATH, '..')
        writers = writers_div_parent.text
        #TODO
        # Talvez o nome dos escritores estejam em outro elemento
    
        album_name = ''
        try:
            album_name = driver.find_element(By.XPATH, "//a[contains(@class, 'PrimaryAlbum')]").text
        except:
            pass
        
        producers = ''
        try:
            producers_div = driver.find_element(By.XPATH, "//p[contains(text(), 'Produced by')]")
            producers_div_parent = producers_div.find_element(By.XPATH, '..')
            producers = producers_div_parent.text
        except:
            pass
        
        print(release_date, writers, album_name, producers)
        
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

Página 1, música número 1.
Release Date
August 18, 2016 Written By
Tiago Iorc & Ana Caetano ANAVITÓRIA (2016) Produced by
Tiago Iorc
Página 1, música número 2.
 Written By
Márcio Borges & Lô Borges Perfil (2006) Produced by
Milton Miranda
Página 1, música número 3.
 Written By
Gilberto Gil & Chico Buarque Chico Buarque (1978) (1978) Produced by
Sérgio de Carvalho
Página 1, música número 4.
 Written By
Tiago Iorc & Ana Caetano ANAVITÓRIA (2016) Produced by
Tiago Iorc
Página 1, música número 5.
Release Date
August 3, 2018 Written By
Tiago Iorc & Ana Caetano O Tempo É Agora (2018) Produced by
Moogie Canazio & Tiago Iorc
Página 1, música número 6.


MaxRetryError: HTTPConnectionPool(host='localhost', port=47887): Max retries exceeded with url: /session/cc16a926757ea8906da912e94d0c7adb/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7efdd5a4f3a0>: Failed to establish a new connection: [Errno 111] Connection refused'))